# Notebook to get datafiles from the server

In [2]:
from astropy.io import fits
import wget
import numpy as np
import random
import os
import pickle
import pandas as pd
import time

In [5]:
area = 'north'
device = 'Astrodisk'

hdulistBricksSouthSummary = fits.open('../../bricks_data/survey-bricks-dr9-south.fits')
data_south = hdulistBricksSouthSummary[1].data
brickname_south = data_south.field('brickname')
brickid_south = data_south.field('brickid')
south_survey_is_south = data_south.field('survey_primary')

hdulistBricksNorthSummary = fits.open('../../bricks_data/survey-bricks-dr9-north.fits')
data_north = hdulistBricksNorthSummary[1].data
brickname_north = data_north.field('brickname')
brickid_north = data_north.field('brickid')
survey_north = data_north.field('survey_primary')
north_survey_is_south = np.invert(survey_north)

In [6]:
brickid_north_south = np.append(brickid_north, brickid_south)
brickname_north_south = np.append(brickname_north, brickname_south)


Begin the actual loop to retrieve a given set of bricks --> split into north  and south for convenience



In [22]:
start = time.time()


print()
print(f"=============================== Download {area} ..... ==================================")
print()

hdulistBricks = fits.open(f'../../bricks_data/survey-bricks-dr9-{area}.fits')
data = hdulistBricks[1].data

bricknames = list(data.field('brickname'))

downloaded_bricks = []

# Getting already downloaded files:
for filename in os.listdir(f'/Volumes/{device}/bricks_data/{area}/'):
    brickn = filename.replace("tractor-", "")
    brickn = brickn.replace(".fits", "")
    downloaded_bricks.append(brickn)


# Getting a random sample of bricknames without replacement and deleting all that are already downloaded
bricknames_sample = [x for x in bricknames if x not in downloaded_bricks]

print(len(bricknames_sample))



=============================== Download north ..... ==================================

48806


In [23]:
df = pd.read_csv(f'../../bricks_data/galaxy_catalogue_{area}.csv', dtype={'BrickID': 'int32', 'LRG': 'int8', 'ELG': 'int8', 'QSO': 'int8'})
brickids_processed = list(df.BrickID.unique())

In [37]:
bricknames_processed = []
for i, id in enumerate(brickids_processed):
    bricknames_processed.append(brickname_north[np.where(brickid_north == id)][0])

print(brickids_processed[3], bricknames_processed[3])

630831 1608p650


In [40]:
bricknames = list(data.field('brickname'))
print(len(bricknames))
print(len(downloaded_bricks))
# Getting a random sample of bricknames without replacement and deleting all that are already downloaded
bricknames_sample = [x for x in bricknames if x not in downloaded_bricks]
# Deleted all those already downloaded

bricknames_sample = [x for x in bricknames_sample if x not in bricknames_processed]

93548
44742


In [50]:
print(len(bricknames_sample))


7623


In [52]:

df_galaxy = pd.DataFrame(columns=['BrickID', 'RA', 'DEC', 'LRG', 'ELG', 'QSO'])
df_stars = pd.DataFrame(columns=['RA', 'DEC', 'GMAG', 'RMAG', 'ZMAG'])

df_galaxy = df_galaxy.append({'BrickID' : 630831, 'RA' : np.nan, 'DEC' : np.nan, 'LRG' : 0, 'ELG':0, 'QSO': 0}, ignore_index=True)
df_galaxy = df_galaxy.append({'BrickID' : 630832, 'RA' : np.nan, 'DEC' : np.nan, 'LRG' : 0, 'ELG':0, 'QSO': 0}, ignore_index=True)
print(df_galaxy.head())

df_galaxy = df_galaxy.append({'BrickID' : 630833, 'RA' : np.nan, 'DEC' : np.nan, 'LRG' : 0, 'ELG':0, 'QSO': 0}, ignore_index=True)

print(df_galaxy.head())
print(df_stars.head())
df_galaxy = df_galaxy.astype(
            {'BrickID': 'int32', 'LRG': 'int8', 'ELG': 'int8', 'QSO': 'int8'})

print(df_galaxy.head())
print(df_stars.head())

    BrickID  RA  DEC  LRG  ELG  QSO
0  630831.0 NaN  NaN  0.0  0.0  0.0
1  630832.0 NaN  NaN  0.0  0.0  0.0
    BrickID  RA  DEC  LRG  ELG  QSO
0  630831.0 NaN  NaN  0.0  0.0  0.0
1  630832.0 NaN  NaN  0.0  0.0  0.0
2  630833.0 NaN  NaN  0.0  0.0  0.0
Empty DataFrame
Columns: [RA, DEC, GMAG, RMAG, ZMAG]
Index: []
   BrickID  RA  DEC  LRG  ELG  QSO
0   630831 NaN  NaN    0    0    0
1   630832 NaN  NaN    0    0    0
2   630833 NaN  NaN    0    0    0
Empty DataFrame
Columns: [RA, DEC, GMAG, RMAG, ZMAG]
Index: []


In [19]:
### South Bricks
start = time.time()

for brick in range(1000):
    randomint = random.randint(0, len(brickname_south))
    brickname = brickname_south[randomint]
    brickid = brickid_south[randomint]

    folder = brickname[:3]
    url = f'https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr9/south/tractor/{folder}/tractor-{brickname}.fits'
    wget.download(url, '/Volumes/Astrostick/bricks_data/south/')


    if (brick % 100) == 0:
        print("Bricks finished:", brick)


print("Time taken: ", time.time() - start)

Bricks finished: 0  of 2000
Bricks finished: 100  of 2000
Bricks finished: 200  of 2000
Bricks finished: 300  of 2000
Bricks finished: 400  of 2000
Bricks finished: 500  of 2000
Bricks finished: 600  of 2000
Bricks finished: 700  of 2000
Bricks finished: 800  of 2000
Bricks finished: 900  of 2000
Bricks finished: 1000  of 2000


OSError: [Errno 28] No space left on device: '/Users/edgareggert/astrostatistics/data_exploration/tractor-0046m020.fitszlzyi8e6.tmp' -> '/Volumes/Astrostick/bricks_data/south//tractor-0046m020.fits'

In [ ]:

start = time.time()

for brick in range(1000):
    randomint = random.randint(0, len(brickname_north))
    brickname = brickname_north[randomint]
    brickid = brickid_north[randomint]

    folder = brickname[:3]
    url = f'https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr9/north/tractor/{folder}/tractor-{brickname}.fits'
    wget.download(url, '/Volumes/Astrostick/bricks_data/north/')

    if (brick % 100) == 0:
        print("Bricks north finished:", brick)


print("Time taken: ", time.time() - start)

### Cleaning up the stick overnight, removing all bricks data


In [22]:
start = time.time()
index = 0
for filename in os.listdir('/Volumes/Astrostick/bricks_data/south/'):
    os.remove(f'/Volumes/Astrostick/bricks_data/south/{filename}')
    index +=1
    if index ==  5:
        break

print("Time taken: ", time.time() - start)

# Time taken for 10:  0.15107083320617676 --> deleting bricks is very fast, hundreds in 1 second, so downloading and processing
# needs to be sped up


#for filename in os.listdir('/Volumes/Astrostick/bricks_data/north/'):
    #os.remove(f'/Volumes/Astrostick/bricks_data/north/{filename}')

Time taken:  0.017904996871948242


In [3]:

area = 'south'
device = 'Astrodisk'
bricks_to_classify = 1500

hdulistBricksSouthSummary = fits.open('../../bricks_data/survey-bricks-dr9-south.fits')
data_south = hdulistBricksSouthSummary[1].data
brickname_south = data_south.field('brickname')
brickid_south = data_south.field('brickid')
south_survey_is_south = data_south.field('survey_primary')

hdulistBricksNorthSummary = fits.open('../../bricks_data/survey-bricks-dr9-north.fits')
data_north = hdulistBricksNorthSummary[1].data
brickname_north = data_north.field('brickname')
brickid_north = data_north.field('brickid')
survey_north = data_north.field('survey_primary')
north_survey_is_south = np.invert(survey_north)

start = time.time()

print()
print(f"=============================== Download {area} ..... ==================================")
print()

hdulistBricks = fits.open(f'../../bricks_data/survey-bricks-dr9-{area}.fits')
data = hdulistBricks[1].data

bricknames = list(data.field('brickname'))
bricks = {}
for brick in bricknames:
    bricks[brick] = 1

downloaded_bricks = []

# Getting already downloaded files:
for filename in os.listdir(f'/Volumes/{device}/bricks_data/{area}/'):
    brickn = filename.replace("tractor-", "")
    brickn = brickn.replace(".fits", "")
    downloaded_bricks.append(brickn)

for brick in downloaded_bricks:
    if brick in bricks:
        bricks.pop(brick)
# Getting a random sample of bricknames without replacement and deleting all that are already downloaded
df = pd.read_csv(f'../../bricks_data/galaxy_catalogue_{area}.csv',
                 dtype={'BrickID': 'int32', 'LRG': 'int8', 'ELG': 'int8', 'QSO': 'int8'})

brickids_processed = list(df.BrickID.unique())
print("Bricks total:", len(bricknames))
print("Bricks processed:", len(brickids_processed))

bricknames_processed = []
for i, id in enumerate(brickids_processed):
    temp = brickname_south[np.where(brickid_south == id)]
    bricknames_processed.append(temp[0])

for brick in bricknames_processed:
    if brick in bricks:
        bricks.pop(brick)

df_galaxy = pd.DataFrame(columns=['BrickID', 'RA', 'DEC', 'LRG', 'ELG', 'QSO'])
df_stars = pd.DataFrame(columns=['RA', 'DEC', 'GMAG', 'RMAG', 'ZMAG'])

print(df_galaxy.head())
print(df_stars.head())

bricknames_sample = list(bricks.keys())
print(f"No of bricks left for area {area}: {len(bricknames_sample)} ")
print("Time taken for: ", round(((time.time() - start) / 60), 2))


=============================== Download south ..... ==================================

Bricks total: 253658
Bricks processed: 103554
Empty DataFrame
Columns: [BrickID, RA, DEC, LRG, ELG, QSO]
Index: []
Empty DataFrame
Columns: [RA, DEC, GMAG, RMAG, ZMAG]
Index: []
No of bricks left for area south: 145414 
Time taken for:  3.55
